In [2]:
import argparse
import os
from functools import partial

import einops
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import tqdm
from huggingface_hub import hf_hub_download
from transformer_lens import HookedTransformer, utils

from attn_sae import *
from sae_training.sae_group import SAEGroup
from sae_training.utils import LMSparseAutoencoderSessionloader
from e2e_sae import SAETransformer

# First Let's See What I have to load for error analysis

In [3]:
import sys
sys.path.append("../")

import error_eval

In [4]:
sae_wes, model_wes = error_eval.load_sae(0)

LanguageModelSAERunnerConfig(model_name='gpt2-small', hook_point='blocks.0.hook_resid_pre', hook_point_layer=0, hook_point_head_index=None, dataset_path='Skylion007/openwebtext', is_dataset_tokenized=False, context_size=128, use_cached_activations=False, cached_activations_path='activations/Skylion007_openwebtext/gpt2-small/blocks.0.hook_resid_pre', d_in=768, n_batches_in_buffer=128, total_training_tokens=300000000, store_batch_size=32, device='mps', seed=42, dtype=torch.float32, b_dec_init_method='geometric_median', expansion_factor=32, from_pretrained_path=None, d_sae=24576, l1_coefficient=8e-05, lp_norm=1, lr=0.0004, lr_scheduler_name=None, lr_warm_up_steps=5000, train_batch_size=4096, use_ghost_grads=True, feature_sampling_window=1000, dead_feature_window=5000, dead_feature_threshold=1e-08, log_to_wandb=True, wandb_project='mats_sae_training_gpt2_small_resid_pre_5', run_name='24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08', wandb_entity=None, wandb_log_frequency=100, n_checkpoints=10, c

/Users/daniellee/Library/Caches/pypoetry/virtualenvs/sae-pathology-s4la4Q7B-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  mps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Dataset is not tokenized! Updating config.
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
Using Ghost Grads.
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
Using Ghost Grads.
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.5

In [4]:
type(sae)

sae_training.sparse_autoencoder.SparseAutoencoder

In [5]:
type(model)

transformer_lens.HookedTransformer.HookedTransformer

# Let's try to load the e2e_sae

7nkdr21r: e2e_ds, L0 of 131, CE of 0.037

In [5]:
model_id = "7nkdr21r"
text = "sparsify/gpt2/" + model_id
sae_transformer_model = SAETransformer.from_wandb(text)

/Users/daniellee/Library/Caches/pypoetry/virtualenvs/sae-pathology-s4la4Q7B-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
model = sae_transformer_model.tlens_model
sae = sae_transformer_model.saes["blocks-6-hook_resid_pre"]

In [32]:
type(sae)

torch.nn.modules.container.ModuleDict

In [39]:
type(sae)

e2e_sae.models.sparsifiers.SAE

In [7]:
device = "mps"

In [8]:
sae = sae.to(device)
model = model.to(device)

Moving model to device:  mps


In [9]:
token_tensor = torch.load("../token_tensor.pt").to(device)

In [ ]:
layer = str(6)
hook_loc = "resid_pre"
pos = None
batch_size = 64

result_df = error_eval.run_error_eval_experiment(
    sae, 
    model, 
    token_tensor, 
    layer, 
    batch_size, 
    pos, 
    hook_loc,
    e2e = True
)

  2%|█▊                                                                                                                   | 4/250 [09:35<13:29:49, 197.52s/it]